In [16]:
import pyomo.environ as pyo

from scipy.spatial.distance import pdist, squareform

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import time
from itertools import cycle

#from pyomo.contrib.appsi.solvers import Highs
import gurobipy
import pandas as pd

In [17]:
model = pyo.ConcreteModel()

In [18]:
N = 100
M = 20
max_timeslots = 16

In [19]:
data = np.array(pd.read_csv('data.csv'))
# data = data[0:2000]
list1 = data.T[0]
list2 = data.T[1]
list1
list2

array([  5,   7,   9, ..., 100,  98, 100], dtype=int64)

# Sets

In [20]:
model.N = pyo.Set(initialize = range(1,N+1))
model.M = pyo.Set(initialize = range(1,M+1))
model.C = pyo.Set(initialize = range(len(list1)))
model.T = pyo.Set(initialize = range(1,max_timeslots+1))

## Parameters

In [21]:
# model.list1 = pyo.Param(range(len(list1)),initialize= list1)
# model.list2 = pyo.Param(range(len(list2)),initialize= list2)

# Variables

In [22]:
model.X = pyo.Var(model.N, model.M, model.T , within=pyo.Binary)
model.Y = pyo.Var(model.M ,model.T, within=pyo.Binary)
model.Z = pyo.Var(model.M , within=pyo.Binary)

## Constraints

In [ ]:
def visit_once_rule(model,i): 
    return sum(model.X[i,j,t] for j in model.M for t in model.T) == 1
model.visit_once_rule = pyo.Constraint(model.N, rule=visit_once_rule)

def event_constraints(model,c,j,t):
    return model.X[list1[c],j,t] + model.X[list2[c],j,t] <= model.Y[j,t]
model.event_constraints = pyo.Constraint(model.C,model.M,model.T, rule=event_constraints)

def one_room_to_event(model,i,j,t): 
    return model.X[i,j,t] <= model.Z[j]
model.one_room_to_event = pyo.Constraint(model.N,model.M, model.T,rule=one_room_to_event)

def one_timeslot_to_event(model,j,t): 
    return sum(model.X[i,j,t] for i in model.N) <= 1
model.one_timeslot_to_event = pyo.Constraint(model.M,model.T, rule=one_timeslot_to_event)

In [ ]:
# model.pprint()

## Obj

In [ ]:
model.obj = pyo.Objective(
    expr=sum(
        model.Z[j]
        for j in model.M) ,
    sense=pyo.minimize,
)


In [ ]:
TIME_LIMIT = 180
THREADS = 4

SOVLER_ENGINE = 'cplex'
#solvers glpk  appsi_highs cplex gurobi gurobi_persistent


solver = pyo.SolverFactory(SOVLER_ENGINE)

if SOVLER_ENGINE == 'cbc':
        solver.options['seconds'] = TIME_LIMIT
elif SOVLER_ENGINE == 'glpk':
        solver.options['tmlim'] = TIME_LIMIT
elif SOVLER_ENGINE == 'appsi_highs':
        solver.options['time_limit'] = TIME_LIMIT
        #solver.options['parallel'] = True
        solver.options['threads'] = THREADS
elif SOVLER_ENGINE == 'cplex':
        solver.options['timelimit'] = TIME_LIMIT
        solver.options['threads'] = THREADS
elif SOVLER_ENGINE == 'gurobi':
        solver.options['timelimit'] = TIME_LIMIT
        solver.options['threads'] = THREADS
elif SOVLER_ENGINE == 'gurobi_persistent':
        solver.options['timelimit'] = TIME_LIMIT
        solver.options['threads'] = THREADS
        solver.set_instance(model) # remove model from  solver.solve(tee=True)
        solver.set_gurobi_param("PoolSolutions", 500)
        solver.set_gurobi_param("PoolSearchMode", 0)

# sol = solver.solve(tee= True)
sol = solver.solve(model, tee= True) #, warmstart=True , logfile= 'log.txt'


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\caleb\AppData\Local\Temp\tmp7pb8s1q1.cplex.log' open.
CPLEX> New value for time limit in seconds: 180
CPLEX> New value for default parallel thread count: 4
CPLEX> Problem 'C:\Users\caleb\AppData\Local\Temp\tmp6tkr3xf0.pyomo.lp' read.
Read time = 1.12 sec. (84.07 ticks)
CPLEX> Problem name         : C:\Users\caleb\AppData\Local\Temp\tmp6tkr3xf0.pyomo.lp
Objective sense      : Minimize
Variables            :   34360  [Binary: 34360]
Objective nonzeros   :      20
Linear constraints   :  880020  [Less: 879920,  Equal: 100]
  Nonzeros           : 2672740
  RHS nonzeros       :     440

Va

In [ ]:
print(sol.solver.status)
print(sol.solver.termination_condition)
print (model.obj())


print ('Total number of rooms used:', sum(model.Z[j].value
                                               for j in model.M))


ok
optimal
6.0
Total number of rooms used: 6.0


In [ ]:
# model.display()

In [ ]:
for t in model.T:
    print ("Time slot:", t)
    time_dict = {}
    for j in model.M:
        for i in model.N:
            if model.X[i,j,t].value == 1:
                time_dict[j] = i
    print (time_dict)

Time slot: 1
{1: 1, 2: 18, 3: 35, 4: 52, 5: 69, 6: 86}
Time slot: 2
{1: 2, 2: 19, 3: 36, 4: 53, 5: 70, 6: 87}
Time slot: 3
{1: 3, 2: 20, 3: 37, 4: 54, 5: 71, 6: 88}
Time slot: 4
{1: 4, 2: 21, 3: 38, 4: 55, 5: 72, 6: 89}
Time slot: 5
{1: 5, 2: 22, 3: 39, 4: 56, 5: 73, 6: 90}
Time slot: 6
{1: 6, 2: 23, 3: 40, 4: 57, 5: 74, 6: 91}
Time slot: 7
{1: 7, 2: 24, 3: 41, 4: 58, 5: 75, 6: 92}
Time slot: 8
{1: 8, 2: 25, 3: 42, 4: 59, 5: 76, 6: 93}
Time slot: 9
{1: 9, 2: 26, 3: 43, 4: 60, 5: 77, 6: 94}
Time slot: 10
{1: 10, 2: 27, 3: 44, 4: 61, 5: 78, 6: 95}
Time slot: 11
{1: 11, 2: 28, 3: 45, 4: 62, 5: 79, 6: 96}
Time slot: 12
{1: 12, 2: 29, 3: 46, 4: 63, 5: 80, 6: 97}
Time slot: 13
{1: 13, 2: 30, 3: 47, 4: 64, 5: 81, 6: 98}
Time slot: 14
{1: 14, 2: 31, 3: 48, 4: 65, 5: 82, 6: 99}
Time slot: 15
{1: 15, 2: 32, 3: 49, 4: 66, 5: 83, 6: 100}
Time slot: 16
{1: 16, 2: 33, 3: 50, 4: 67, 5: 84}
Time slot: 17
{1: 17, 2: 34, 3: 51, 4: 68, 5: 85}


In [ ]:
# Events per room

for j in model.M:
    events = sum(model.X[i,j,t].value for i in model.N for t in model.T)
    if events >0:
        print ("Room Number:", j)
        print ("Number of events:", events, '\n')

Room Number: 1
Number of events: 17.0 

Room Number: 2
Number of events: 17.0 

Room Number: 3
Number of events: 17.0 

Room Number: 4
Number of events: 17.0 

Room Number: 5
Number of events: 17.0 

Room Number: 6
Number of events: 15.0 

